In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write

In [3]:
#### AVOID folding of output cell 

In [4]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [5]:
data_dump = WidgetDataDumper(prefix="ex_04")
display(data_dump)

WidgetDataDumper(children=(Text(value='', description='Name'), Button(description='Save all', style=ButtonStyl…

_Reference textbook / figure credits: Allen, Tildesley, Computer simulations of liquids, (2017), Chapter 1_

# Interatomic potentials

TODO MC - an intro to potentials
* Show typical example with Coulomb, dispersion, bonded terms, ....
* Discuss these in terms of short and long range, bonded and non-bonded

An archetypal example of a non-bonded potential is the Lennard-Jones potential (if you are curious, you can read [the paper in which the general functional form was proposed](https://doi.org/10.1098/rspa.1924.0081)).
The LJ potential is a non-bonded pair potential $V(r)$ in which the attractive and repulsive parts are both algebraic functions of the interatomic separation, $A/r^m-B/r^n$. Usually $1/r^6$ is used for the attractive part (that physically corresponds to dispersion/van der Waals forces), and $1/r^{12}$ for the repulsive parts (which is chosen just to have a steep repulsive wall, and because back in the old days you could compute this just by squaring $1/r^6$, which was cheaper than recomputing another power. 

You can experiment below with the more general form of the potential,
$$
V(r) = \frac{A}{r^m} - \frac{B}{r^n}
$$
See how exponents and prefactors change the shape of the curve.

In [6]:
def plot_LJ(ax, A, B, m, n, x_max = 3, y_min_relative = -1.5, y_max_relative = 2, n_points = 200):
    
    if (m != n):
        # min_pos and min_energy are max pos and max energy when n > m
        min_pos = np.exp((np.log(A) - np.log(B) + np.log(m) - np.log(n))/(m - n))
        min_energy = A / (min_pos ** m) - B / (min_pos ** n)
        min_energy = np.abs(min_energy)

        y_min = min_energy * y_min_relative
        y_max = min_energy * y_max_relative
    else:
        y_min, y_max = y_min_relative, y_max_relative
        
    grid = np.linspace(0, x_max, 200)[1:] # excluding 0
    curve = A / (grid ** m) - B / (grid ** n)
    
    ax.plot(grid, curve, color = 'black', linewidth = 2)
    ax.set_title(r"$V(r) = \frac{A}{r^m} - \frac{B}{r^n}$", fontsize = 15)
    ax.set_xlim([0, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_xlabel("r", fontsize = 15)
    ax.set_ylabel("V(r)", fontsize = 15)
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)

In [7]:
A = WidgetPlot(plot_LJ, WidgetParbox(A = (1.0, 0.0, 10, 0.1, r'A'),
                                       B = (1.0, 0.0, 10, 0.1, r'B'),
                                       m = (12, 1, 20, 1, r'm'),
                                       n = (6, 1, 20, 1, r'n'),
                                       ))

In [8]:
display(A)

WidgetPlot(children=(WidgetParbox(children=(FloatSlider(value=1.0, continuous_update=False, description='A', l…

The more common form to express the LJ potential is

$$
V(r) = 4\epsilon \left((\frac{\sigma}{r})^12 - (\frac{\sigma}{r})^6\right).
$$

<span style="color:blue">**01** Compute analytically the equilibrium separation $r_0$ between two atoms (i.e. the position of the minimum in the $V(r)$ curve. What is the corresponding energy? </span>

In [9]:
ex01_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)

Textarea(value='enter your answer', layout=Layout(width='100%'))

<span style="color:blue">**02** Now consider a set of four atoms arranged as a square with side $a$. Write a function that computes the total LJ potential for this structure, as a function of $a$. Inspect the curve as a function of $a$.</span>

_Take for simplicity $\epsilon=1$ and $\sigma=1$ (which is equivalent to writing the problem in natural units. You can write the summation as a sum over the pair distances, without writing explicitly the position of the particles._

In [10]:
# set upt the code widget window
ex02_wci = WidgetCodeInput(
        function_name="total_LJ_square", 
        function_parameters="a",
        docstring="""
Computes the total LJ potential for the structure of four atoms arranged as a square with side a. 

:param a: side of the square
        
:return: the value of the total energy
""",
        function_body="""
# Write your solution. You can use np.sqrt(2) to get the value of sqrt(2)

import numpy as np

def compute_LJ(r):
    # computes the value of LJ potential depending on the distance r
    
    return 4 * (1 / (r ** 12) - 1 / (r ** 6))
    
    
return 0.0
"""
        )

data_dump.register_field("ex02-function", ex02_wci, "function_body")

In [11]:
def compute_LJ(r):
    return 4 * (1 / (r ** 12) - 1 / (r ** 6))
def reference_func_02(a):
    import numpy as np
    return 4 * compute_LJ(a) + 2 * compute_LJ(np.sqrt(2.0) * a)

def match_energy(first, second, epsilon = 1e-8):
    return abs(first - second) < 1e-8


In [12]:
def plot_total_energy(ax, x_max, y_min, y_max, n_points = 200):
    grid = np.linspace(0.0, x_max, n_points)[1:]
    func = ex02_wci.get_function_object()    
    values = [func(x) for x in grid]
    ax.plot(grid, values, color = 'black', linewidth = 2)
    ax.set_xlim(0.0, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel("a", fontsize = 15)
    ax.set_ylabel("total energy", fontsize = 15)
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    

In [13]:
ex_02_plot = WidgetPlot(plot_total_energy, WidgetParbox(x_max = (4.0, 1.0, 20, 0.1, r'$x_{max}$'),
                                       y_min = (-1.0, -10, 0, 0.1, r'$y_{min}$'),
                                       y_max = (2.0, 0, 10, 0.1, r'$y_{max}$'),
                                       ));

In [14]:
ex_02_ref_values = {(value, ) : reference_func_02(value) for value in np.linspace(0.1, 5, 20)}
ex02_wcc = WidgetCodeCheck(ex02_wci, ref_values = ex_02_ref_values, ref_match = match_energy, demo=ex_02_plot)    
display(ex02_wcc)

WidgetCodeCheck(children=(WidgetCodeInput(code_theme='nord', docstring='\nComputes the total LJ potential for …

In [15]:
# TODO SP: create a code widget and a stub of the function - they should define V(r) inside the 
# function and sum manually over the 4 a and two sqrt(2) a distances. the widget should allow 
# them to zoom onto a portion of the x axis, just by moving two sliders to select the range

<span style="color:blue">**03** Is the equilibrium separation between the particles the same as that minimizing the energy of a dimer? Can you write both the numerical and analytical values?</span>

In [ ]:
ex03_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex03-answer", ex03_txt, "value")
display(ex03_txt)

## ex05

In [ ]:
lj55 = read('data/lj-structures.xyz',":1")

In [ ]:
ex05_coordinates = []
for position in lj55[0].get_positions():
    ex05_coordinates.append(tuple(position))
    
ex05_coordinates = tuple(ex05_coordinates)

# set upt the code widget window
ex05_wci = WidgetCodeInput(
        function_name="total_LJ_icosahedral", 
        function_parameters="coordinates, r_cut",
        docstring="""
Computes total LJ energy of the icosahedral cluster

:param coordinates: Cartesian coordinates of the atoms stored as a nested tuple. 
coordinates[i][0], coordinates[i][1] and coordinates[i][2] are x, y, and z coordinates of i-th atom correspondingly.
:param r_cut: cutoff distance
        
:return: total LJ energy of the icosahedral cluster
""",
        function_body="""
        
import numpy as np

def compute_LJ(r):
    return 4 * (1.0 / (r ** 12) - 1.0 / (r ** 6))

def compute_distance(first, second):
    total = 0.0
    for i in range(3):
        total += (first[i] - second[i]) ** 2
    return np.sqrt(total)

total = 0.0
for i in range(len(coordinates)):
    for j in range(i + 1, len(coordinates)):
        distance = compute_distance(coordinates[i], coordinates[j])
        if distance < r_cut:
            total += compute_LJ(distance)
return total
"""
        )

data_dump.register_field("ex05-function", ex05_wci, "function_body")

def plot_icosahedral_energy(ax, x_max, y_min, y_max, n_points = 200):
    grid = np.linspace(0, x_max, n_points)
    func = ex05_wci.get_function_object()    
    values = [func(lj55[0].get_positions(), x) for x in grid]
    
    ax.plot(grid, values, color = 'red', linewidth = 2)
    ax.set_xlim(0, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel("r_cut", fontsize = 15)
    ax.set_ylabel("total energy", fontsize = 15)
    ax.set_title("icosahedral cluster")
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    
ex05_plot = WidgetPlot(plot_icosahedral_energy, WidgetParbox(x_max = (4.0, 1.0, 10, 0.1, r'$x_{max}$'),
                                       y_min = (-500.0, -1000, 0, 1, r'$y_{min}$'),
                                       y_max = (0, 200, 1000, 1, r'$y_{max}$'),
                                       ));


def match_energy(first, second, epsilon = 1e-5):
    return abs(first - second) < 1e-5

def reference_func_05(coordinates, r_cut):
    def compute_LJ(r):
        return 4 * (1.0 / (r ** 12) - 1.0 / (r ** 6))
    
    def compute_distance(first, second):
        total = 0.0
        for i in range(3):
            total += (first[i] - second[i]) ** 2
        return np.sqrt(total)
    
    total = 0.0
    for i in range(len(coordinates)):
        for j in range(i + 1, len(coordinates)):
            distance = compute_distance(coordinates[i], coordinates[j])
            if distance < r_cut:
                total += compute_LJ(distance)
    return total



ex_05_ref_values = {(ex05_coordinates, value, ) : reference_func_05(ex05_coordinates, value)
                    for value in np.linspace(0, 10, 50)}

ex05_wcc = WidgetCodeCheck(ex05_wci, ref_values = ex_05_ref_values, ref_match = match_energy, demo=ex05_plot)       
display(ex05_wcc)

# TESTS & CRAP

In [ ]:
frame = ase.Atoms("He2")

In [ ]:
frame.positions[:,0] = [-1,1]

In [ ]:
from ase.calculators import lj, eam

In [ ]:
ljcalc = lj.LennardJones(sigma=1.0, epsilon=1.0)

In [ ]:
frame.calc = ljcalc
frame.get_potential_energy()
frame.get_forces()

In [ ]:
frame.info

In [ ]:
from ase.build import bulk
frame = bulk('Al', 'fcc', a=4.05, cubic=True)

In [ ]:
eamcalc = eam.EAM(potential='data/Al99.eam.alloy')

In [ ]:
frame.calc = eamcalc
frame.get_potential_energy()

# x